# **Set according to environment (e.g. local, Google Colab...)**

In [1]:
project_folder = ''

# **Body**

In [2]:
from custom_libraries.miscellaneous import *
from custom_libraries.image_dataset import *
import numpy as np

KeyboardInterrupt: 

In [ ]:


class treeLayer(tf.keras.layers.Layer):

    def __init__(self, Input_size=3072, Activation="relu"):
        super(treeLayer, self).__init__()
        self.Input_size = Input_size
        self.Activation = Activation

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(1, self.Input_size),
                                      initializer=tf.keras.initializers.HeNormal,
                                      trainable=True)
        self.summer = np.zeros((self.Input_size, self.Input_size // 2))
        for i in range(self.Input_size):
            self.summer[i, i // 2] = 1

        self.summer = tf.convert_to_tensor(self.summer, dtype=tf.float32)

    def call(self, inputs):
        x = tf.math.multiply(inputs, self.kernel)
        x = tf.matmul(x, self.summer)
        x = tf.nn.leaky_relu(x, alpha=.01)
        return x


def create_model(input_size, num_trees=1):
    model = tf.keras.Sequential()
    while input_size > num_trees:
        model.add(treeLayer(input_size))
        input_size = input_size // 2
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', use_bias=False))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(
                      reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
                  ),
                  metrics=[tf.keras.losses.BinaryCrossentropy(name='binary_crossentropy'), 'acc'])

    return model


In [ ]:
import gc

# Initialize settings
bs = 256
trials = 10
epochs = 2000
trees_set = [1]

# Load class-dataset list
# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

classes = [[3, 5, 'mnist'],
           [0, 6, 'fmnist'],
           [2, 6, 'kmnist'],
           [14, 17, 'emnist']]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60)

#history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
history = np.zeros((len(classes), trials, len(trees_set), 2))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    test_ds = ImageDataset(ds, 'test', data_dir=None)
    train_ds = ImageDataset(ds, 'train', data_dir=None)

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")
        X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                             validation_size=len(test_ds.images))
        X_test, y_test = test_ds.subset()
        test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(
            lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

        #if history[j, -1, k, 0] != 0:
        #  continue

        for i in range(trials):
            print(f"Trial {i + 1}")

            #with tf.device('/device:GPU:0'):

            model = create_model(input_size=X_train[i].shape[1] * trees, num_trees=trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

            fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                    validation_data=valid_set, validation_batch_size=bs,
                                    callbacks=[callback], verbose=0)
            print_fit_history(fit_history, epochs)

            evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
            print_evaluate_history(evaluate_history)

            history[j, i, k] = evaluate_history[1:]

            np.save(project_folder + 'results/ktree_history.npy', history,
                    allow_pickle=True)

            del model, train_set, valid_set
            gc.collect()

In [ ]:
history = np.load(project_folder + 'results/ktree_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{k}-tree")
        print(f"Accuracy: mean = {round(np.mean(history[j,:,k,1]), 4)}, standard deviation = {round(np.std(history[j,:,k,1]), 4)}")